# Final Project: House Price Prediction
## Corpus Christi Team
### Step 4b (Save crime data to the PostgreSQL Data Base)

In [1]:
# Import libraries
import psycopg2
import psycopg2.extras
import time
import re
import os
import glob
import csv
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from config import db as dbpwd

In [2]:
# Data base connection details
hostname = 'localhost'
database = 'real_estate_data_test'
username = 'postgres'
pwd = dbpwd
port_id = 5432
conn = None
cur = None

In [3]:
# Define the input data path and input file name
data_path = "../01_Gathering/data/*.csv"
listing = glob.glob(data_path)

for file in listing:
    if 'Houston_TX_Crime_Stats_per_ZipCode' in file:
        input_file = file
        print(f'The input file is {file}')

# Define the table name to be saved in the DB
regex = "(Crime.+).csv"
match = re.findall(rf"{regex}", input_file)
name = ''.join(match)
table_name = 'htx_' + name.lower()
print(f'The input file will be saved to the db as a table named: {table_name}')

The input file is ../01_Gathering/data\Houston_TX_Crime_Stats_per_ZipCode_2022.csv
The input file will be saved to the db as a table named: htx_crime_stats_per_zipcode_2022


In [4]:
# Convert the input data to a pandas DF for QC
df = pd.read_csv(input_file)
df.head(3)

,zip,crimeCount,crimePercentage
0,77002,5300,2.306705
1,77003,2597,1.130285
2,77004,5647,2.457729


#### Save the table to the DB

In [5]:
try:
    #connect to the database
    conn = psycopg2.connect(host=hostname,
                           dbname=database,
                           user=username,
                           password=pwd,
                           port=port_id)  

    #create a cursor object: it is used to interact with the database
    cur = conn.cursor()
    
    # Remove the table only if iti exists
    cur.execute('DROP TABLE IF EXISTS {}'.format(table_name))

    #create a DB table with same headers as csv file
    create_script = '''CREATE TABLE IF NOT EXISTS {} (
    zipCode INT PRIMARY KEY,
    crimeCount INT,
    crimePercentage FLOAT)'''.format(table_name)
    
    # Execute and commit the changes
    cur.execute(create_script)
    
    # Insert Data into the DB table
    insert_script = '''INSERT INTO {} (
    zipCode,
    crimeCount,
    crimePercentage)
    VALUES (%s, %s, %s)'''.format(table_name)
    
    with open(input_file, 'r') as csvfile:
        dataReader = csv.reader(csvfile)
        next(dataReader)
        for row in dataReader:
            zipCode = row[0]
            crimeCount = row[1]
            crimePercentage = row[2]

            values = [zipCode, crimeCount, crimePercentage]
        
            cur.execute(insert_script, values)
    
    conn.commit()

except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()
        print(f'Table {table_name} was succesfully saved to the db {database}')

Table htx_crime_stats_per_zipcode_2022 was succesfully saved to the db real_estate_data_test


#### Read the table from the DB

In [6]:
with psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(hostname, port_id, database, username, pwd)) as conn:
    sql = "select * from {};".format(table_name)
    db2df = pd.read_sql_query(sql, conn)

db2df.head(3)

,zipcode,crimecount,crimepercentage
0,77002,5300,2.306705
1,77003,2597,1.130285
2,77004,5647,2.457729
